## Attention Scoring Function

In [2]:
import math
import torch
from torch import nn
from d2l import torch as d2l

### Attention Scoring Function
Define the following
* Attention scoring function (or scoring function)
* attention weights
* attentionl pooling function

### Masked Softmax Operation

What's the purpose of masked softmax attention?

In [3]:
def masked_softmax(X, valid_lens):
    """Perform softmax operation by masking elements on the last axis."""
    # `X`: 3D tensor, `valid_lens`: 1D or 2D tensor
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            valid_lens = valid_lens.reshape(-1)
        # On the last axis, replace masked elements with a very large negative
        # value, whose exponentiation outputs 0
        X = d2l.sequence_mask(X.reshape(-1, shape[-1]), valid_lens,
                              value=-1e6)
        return nn.functional.softmax(X.reshape(shape), dim=-1)

If valid_lens=None no masking is done

In [4]:
A = torch.rand(2, 2, 4)
B = None 

masked_softmax(A, B)

tensor([[[0.3249, 0.2135, 0.2836, 0.1780],
         [0.1865, 0.2427, 0.2788, 0.2920]],

        [[0.2279, 0.2721, 0.2625, 0.2375],
         [0.2796, 0.1986, 0.3176, 0.2041]]])

In the following A has minibatch of two 2x4 examples. The valid lengths for each example is 2 and 3. Therefore the softmax is applied to the sequence curtailed to that length. Length is the last dimension of the example. 

In [5]:
A = torch.rand(2, 2, 4)
B = torch.tensor([2, 3])

masked_softmax(A, B)

tensor([[[0.4931, 0.5069, 0.0000, 0.0000],
         [0.6444, 0.3556, 0.0000, 0.0000]],

        [[0.3168, 0.2561, 0.4271, 0.0000],
         [0.3411, 0.4266, 0.2323, 0.0000]]])

In the following example the valid lens is a 2D tensor

In [6]:
A = torch.rand(2, 2, 4)
B = torch.tensor([[1, 3], [2, 4]])
masked_softmax(A, B)

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.3036, 0.3649, 0.3315, 0.0000]],

        [[0.4729, 0.5271, 0.0000, 0.0000],
         [0.1656, 0.3407, 0.2565, 0.2372]]])

### Additive Attention
What does additive attention do when the query $\mathbf{q}$ 
and the key $\mathbf{k}$ are of different lengths

$$a(\mathbf q, \mathbf k) = \mathbf w_v^\top \text{tanh}(\mathbf W_q\mathbf q + \mathbf W_k \mathbf k) \in \mathbb{R},$$

* What are the learnable parameters?
* What is the purpose of Dropout here?

In [7]:
#@save
class AdditiveAttention(nn.Module):
    def __init__(self, key_size, query_size, num_hiddens, dropout, **kwargs):
        super(AdditiveAttention, self).__init__(**kwargs)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=False)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=False)
        self.w_v = nn.Linear(num_hiddens, 1, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, queries, keys, values, valid_lens):
        queries, keys = self.W_q(queries), self.W_k(keys)
        # After dimension expansion, shape of `queries`: (`batch_size`, no. of
        # queries, 1, `num_hiddens`) and shape of `keys`: (`batch_size`, 1,
        # no. of key-value pairs, `num_hiddens`). Sum them up with
        # broadcasting
        features = queries.unsqueeze(2) + keys.unsqueeze(1)
        features = torch.tanh(features)
        # There is only one output of `self.w_v`, so we remove the last
        # one-dimensional entry from the shape. Shape of `scores`:
        # (`batch_size`, no. of queries, no. of key-value pairs)
        scores = self.w_v(features).squeeze(-1)
        self.attention_weights = masked_softmax(scores, valid_lens)
        # Shape of `values`: (`batch_size`, no. of key-value pairs, value
        # dimension)
        return torch.bmm(self.dropout(self.attention_weights), values)

In [8]:
queries, keys = torch.normal(0, 1, (2, 1, 20)), torch.ones((2, 10, 2))
# The two value matrices in the `values` minibatch are identical
values = torch.arange(40, dtype=torch.float32).reshape(1, 10,
                                                       4).repeat(2, 1, 1)
valid_lens = torch.tensor([2, 6])

attention = AdditiveAttention(key_size=2, query_size=20, num_hiddens=8,
                              dropout=0.1)
attention.eval()
result = attention(queries, keys, values, valid_lens)

In [9]:
result

tensor([[[ 2.0000,  3.0000,  4.0000,  5.0000]],

        [[10.0000, 11.0000, 12.0000, 13.0000]]], grad_fn=<BmmBackward0>)

In [10]:
print("Shape")
pp = lambda x, y: print("{:15}: {}".format(x, y.shape))

pp("Keys (K)", keys)
pp("Values (V)", values)
pp("Queries (Q)", queries)
pp("Valid Lens", valid_lens)
pp("Result (F)", result)

Shape
Keys (K)       : torch.Size([2, 10, 2])
Values (V)     : torch.Size([2, 10, 4])
Queries (Q)    : torch.Size([2, 1, 20])
Valid Lens     : torch.Size([2])
Result         : torch.Size([2, 1, 4])


**Shapes**

* .shape[0] of all is batch size (= 2)
* K.shape[1] == V.shape[1] = number of keys/values
* Third dimension of Q, K, V are different (that's allowed by this attention model)
* F.shape[1] == Q.shape[1] = number of queries
* F.shape[2] == V.shape[2] = pooled output shape

In [ ]:
k